### Imports

In [ ]:
import chipwhisperer as cw
import matplotlib.pyplot as plt
import numpy as np
import time
import struct
import random

from scipy.signal import find_peaks

### Function Definitions

In [ ]:
def random_float(min_val, max_val):
    # Generate a random float between min_val and max_val
    rand_float = random.uniform(min_val, max_val)
    # Round to 2 decimal places
    return round(rand_float, 2)

In [ ]:
def float_to_bytearray_32bit_little_edian(f):
    # Pack the float as a 32-bit (4-byte) IEEE 754 floating point number
    packed = struct.pack('f', f)
    # Convert to bytearray
    return bytearray(packed)

In [ ]:
def scope_setup(samples=24431, decimate=1):
    # arm the scope
    scope.arm()
    
    # Set the maximum number of points in a trace
    scope.adc.fifo_fill_mode = "normal"
    scope.adc.samples = samples
    scope.adc.decimate = decimate

In [ ]:
def capture_trace(cmd_data, cmd='p', scmd=128, prints=True):
    scope.arm()
    # flush the UART buffer
    target.flush()
    
    target.send_cmd(cmd, scmd, cmd_data)
    ret = scope.capture()
    trace = scope.get_last_trace()
    
    returned_data = target.read_cmd('r')
    ack = target.read_cmd('e')
    if prints:
        print(f'r\t- target.read_cmd("r"):\t{returned_data}')
        print(f'ack\t- target.read_cmd("e"):\t{ack}')
    return trace
    

In [ ]:
def plot_trace(trace, vlines=[], save=False, fig_name='figure.svg'):
    # Plot the trace
    plt.plot(trace, linewidth=0.8)
    if len(vlines) > 0:
        plt.vlines(vlines, -0.3, 0.2, colors='red')
    if save:
        plt.savefig(fig_name, format='svg')
    plt.show()

In [ ]:
def filter_peaks(arr, old_idxs, level):
    
    if level > 0: 
        a = np.array(arr, dtype=np.float32)
        indices, props = find_peaks(a)
        
        peak_arr = []
        new_idxs = []
        for idx in indices:
            peak_arr.append(arr[idx])
            new_idxs.append(old_idxs[idx])
        
        filtered_indices = filter_peaks(peak_arr, new_idxs, level - 1)
        return filtered_indices
    else:
        return old_idxs

In [ ]:
def find_local_maxima(trace, levels):
    original_indices = []
    for i in range(len(trace)):
        original_indices.append(i)
    return filter_peaks(trace, original_indices, levels)
    


### Target Setup

In [ ]:
#Scope setup
scope = cw.scope()
scope.default_setup()

target = cw.target(scope, cw.targets.SimpleSerial2) #cw.targets.SimpleSerial can be omitted
#MY CHANGES - changed target to SimpleSerial2 - to be able to send_cmd

In [ ]:
scope_setup(samples=24430, decimate=1)

In [ ]:
%%bash
cd version_02/
make PLATFORM='CWLITEARM' CRYPTO_TARGET=NONE

In [ ]:
cw.program_target(scope, cw.programmers.STM32FProgrammer, "version_02/simpleserial-target-CWLITEARM.hex")

### Trace Capture Test

In [ ]:
float_val = -0.213222
float_bytearray = float_to_bytearray_32bit_little_edian(float_val)

print(float_bytearray)

trace_wave = capture_trace(float_bytearray, scmd=0)
trace = cw.Trace(wave=trace_wave, textin=float_val, textout=None, key=None)

In [ ]:
peak_indices = find_local_maxima(trace_wave, 6)
print(len(peak_indices))
print(peak_indices)

In [ ]:
plot_trace(trace_wave, vlines=peak_indices)


In [ ]:
plot_trace(trace_wave[:19650], vlines=[], save=False, fig_name="./trace_figures/full_network_dummy_ops_last_layer_cutoff.svg")

In [ ]:
full_network = trace_wave[:19650]
print(len(full_network))
try: 
    f = open("./trace_figures/full_network.txt", "w")
    f.write("x y\n")
    for i in range(len(full_network)):
        f.write(f'{i} {full_network[i]}\n')
    f.close()
except(error):
    print(f'Failed to write full network to file - {error}')
    

### Initialize the project

The Chipwhisperer `Project` class can be used to keep a collection of traces. 

In [ ]:
proj_dir = "./cw_projects/"
proj_name = "deranged_protected_w5_random_inputs"
scmd_val = 1

#project-01
num_traces = 10000
min_in_val = -2
max_in_val = 2

In [ ]:
proj = cw.create_project(proj_dir + proj_name)

In [ ]:
input_vals = []

for i in range(num_traces):
    input_vals.append(random.uniform(min_in_val, max_in_val))


print(max(input_vals))
print(min(input_vals))
print(len(input_vals))

In [ ]:
start = time.time()
completed_counter = 0
for i in range(num_traces):
    cmd_data = float_to_bytearray_32bit_little_edian(input_vals[i])
    
    trace_wave = capture_trace(cmd_data=cmd_data, scmd=scmd_val, prints=False)
    trace = cw.Trace(wave=trace_wave, textin=input_vals[i], textout=None, key=None)
    proj.traces.append(trace)
    
    completed_counter += 1
    if completed_counter % 100 == 0: 
        print(f'completed {completed_counter} traces in\t{time.time() - start} seconds' )

end = time.time()
print(f'capturing traces finished in {end - start} seconds!')

In [ ]:
proj.save()
proj.close()

In [ ]:
proj = cw.open_project(proj_dir + proj_name)

In [ ]:
print(len(proj.traces))

In [ ]:
proj.close()